In [1]:
import pandas as pd
import numpy as np
import chromedriver_autoinstaller
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = f"https://www.yelo.by/location/%D0%9C%D0%B8%D0%BD%D1%81%D0%BA"
            
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # 시크릿 모드
chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
try: # 크롬 드라이버
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        # WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
        # driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')


In [ ]:
df = pd.DataFrame()
for idx in tqdm(range(1, 576)):
    try:
        if idx == 1:
            driver.get(url)
        else: driver.get(url + f"/{idx}")
        href_lst = driver.find_element(By.XPATH, '//*[@id="listings"]')
        company_link_lst = []
        for hrefs in href_lst.find_elements(By.TAG_NAME, "a"):
            href = hrefs.get_attribute("href")
            if "company" in href and "reviews" not in href and href not in company_link_lst:
                company_link_lst.append(href)
        for company_link in company_link_lst:
            driver.get(company_link)
            div_lst = driver.find_elements(By.TAG_NAME, "div")
            lst = []
            for div in div_lst:
                if div.get_attribute('class') == 'info': lst.append(div.text)
            company_name = lst[0].split("\n")[1]
            append_df = pd.DataFrame({company_name : lst[1:]})
            df = pd.concat([df, append_df], axis=1)
            print(f"appended {company_name}")
    except:
        print("Nahh")

In [4]:
df.to_csv("벨라루스.csv", index=False)

In [ ]:
company_lst = list(df.columns)
new_df = pd.DataFrame()
fail_lst = []
for row in company_lst:
    column_dict = {}
    for idx in range(len(df[row])):
        column_dict["COMPANY NAME"] = row
        try:
            if "ADDRESS\n" in df[row][idx]: column_dict["ADDRESS"] = df[row][idx].replace("ADDRESS\n", "")
            elif "PHONE NUMBER\n" in df[row][idx]: column_dict["PHONE NUMBER"] = df[row][idx].replace("PHONE NUMBER\n", "")
            elif "MOBILE PHONE\n" in df[row][idx]: column_dict["MOBILE PHONE"] = df[row][idx].replace("MOBILE PHONE\n", "")
            elif "WEBSITE\n" in df[row][idx]: column_dict["WEBSITE"] = df[row][idx].replace("WEBSITE\n", "")
            elif "ESTABLISHMENT YEAR" in df[row][idx]: column_dict["ESTABLISHMENT YEAR"] = df[row][idx].split()[-1]
            elif "EMPLOYEES" in df[row][idx]: column_dict["EMPLOYEES"] = df[row][idx].split()[-1]
            elif "COMPANY MANAGER" in df[row][idx]: column_dict["COMPANY MANAGER"] = "".join(df[row][idx].split()[2:])
            elif "EMAIL" in df[row][idx]: column_dict["EMAIL"] = df[row][idx].replace("EMAIL\n", "")
            elif "DESCRIPTION\n" in df[row][idx]: column_dict["DESCRIPTION"] = df[row][idx].replace("DESCRIPTION\n", "")
            elif "WORKING HOURS\n" in df[row][idx]: column_dict["WORKING HOURS"] = df[row][idx].replace("WORKING HOURS\n", "")
            elif "PRODUCTS & SERVICES\n" in df[row][idx]: column_dict["PRODUCTS & SERVICES"] = df[row][idx].replace("PRODUCTS & SERVICES\n", "")
            elif "LISTED IN CATEGORIES\n" in df[row][idx]: column_dict["LISTED IN CATEGORIES"] = df[row][idx].replace("LISTED IN CATEGORIES\n", "")
            elif "KEYWORDS\n" in df[row][idx]: column_dict["KEYWORDS"] = df[row][idx].replace("KEYWORDS\n", "")
            elif "Related Companies to" in df[row][idx]: column_dict["Related Companies"] = df[row][idx].split()[2:]
            elif "CONTACT PERSON\n" in df[row][idx]: column_dict["CONTACT PERSON"] = df[row][idx].replace("CONTACT PERSON\n", "")
            elif "FAX\n" in df[row][idx]: column_dict["FAX"] = df[row][idx].replace("FAX\n", "")
            elif "REGISTRATION CODE" in df[row][idx]: column_dict["REGISTRATION CODE"] = df[row][idx].split()[-1]
            elif "VAT REGISTRATION" in df[row][idx]: column_dict["VAT REGISTRATION"] = df[row][idx].split()[-1]
            else: 
                if "nan" not in df[row][idx]:
                    print(df[row][idx])
        except:
            pass
    new_df = new_df.append(column_dict, ignore_index=True)

In [ ]:
new_df.to_csv("3차년도")